<a href="https://colab.research.google.com/github/cedric-bit/Geospatial-Data-Science-2023/blob/main/loadDATA509.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install gtfs-realtime-bindings


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install gtfs_kit protobuf gtfs-realtime-bindings pandas folium geopandas osmnx






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 45.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.6 MB/s eta 0:00:00
  

In [5]:
!pip install --upgrade protobuf



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 KB 21.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.1 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.1 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 4.22.1 which is incompatible.


In [6]:
from google.colab import drive
drive.mount('/content/drive')
import os
from datetime import datetime

shortcut_path = '/content/drive/MyDrive/Geospatial Data Science 2023' 
geospacialdata_folder_name = "Geospatial Data Science 2023"

# Vérifiez si le nom du dossier est présent dans le chemin du raccourci
if geospacialdata_folder_name in shortcut_path:
    base_path = shortcut_path
else:
    for folder in os.listdir(shortcut_path):
        if folder == geospacialdata_folder_name:
            base_path = os.path.join(shortcut_path, geospacialdata_folder_name)
            break

if base_path is not None:
    print(f"Dossier geospacialdata trouvé: {base_path}")
else:
    print("Dossier geospacialdata non trouvé")

# Liste les fichiers et les dossiers dans le répertoire Geospatial Data Science 2023
#for item in os.listdir(base_path):
 #   item_path = os.path.join(base_path, item)
  #  if os.path.isdir(item_path):
   #     print(f"Dossier: {item}")

   
# Sélectionnez le dossier 2023-03-20
selected_folder = "2023-03-20"
selected_folder_path = os.path.join(base_path, selected_folder)

# Affichez le contenu du dossier sélectionné
for item in os.listdir(selected_folder_path):
    item_path = os.path.join(selected_folder_path, item)
    if os.path.isdir(item_path):
        print(f"Dossier: {item}")
  #  elif os.path.isfile(item_path):
   #     print(f"Fichier: {item}")




def check_file_format(file_path):
    with open(file_path, 'rb') as f:
        signature = f.read(2)

    if signature == b'\x1f\x8b':
        print(f"{file_path} est un fichier gzip.")
    else:
        print(f"{file_path} n'est pas un fichier gzip.")

# Utilisez cette fonction pour vérifier le format d'un fichier .gtfsrt spécifique
for gtfsrt_file in os.listdir(selected_folder_path):
    gtfsrt_file_path = os.path.join(selected_folder_path, gtfsrt_file)
    #check_file_format(gtfsrt_file_path)

#from google.transit import gtfs_realtime_pb2



for gtfsrt_file in os.listdir(selected_folder_path):
    if gtfsrt_file.endswith('.gtfsrt'):
        gtfsrt_file_path = os.path.join(selected_folder_path, gtfsrt_file)
       # process_gtfsrt_file(gtfsrt_file_path)


import osmnx as ox


from google.transit import gtfs_realtime_pb2
from google.protobuf import text_format



def process_gtfsrt_file(file_path):
    feed = gtfs_realtime_pb2.FeedMessage()
    
    try:
        with open(file_path, 'rb') as f:
            content = f.read()
            feed.ParseFromString(content)
    except Exception as e:
        #print(f"Erreur lors du traitement du fichier {file_path}: {e}")
        return None

    return feed


feeds = []
#Ce code extrait les stop_time_update de chaque fichier GTFS Realtime et les ajoute à une liste stop_time_updates_list

def extract_stop_time_updates(feed):
    if feed is None:
        return []

    stop_time_updates = []

    for entity in feed.entity:
        if entity.HasField('trip_update'):
            trip_update = entity.trip_update
            for stop_time_update in trip_update.stop_time_update:
                stop_time_updates.append(stop_time_update)

    return stop_time_updates

feeds = []
first_file_processed = False
# a la base pour imprimer juste le contenue du premier fichier
for gtfsrt_file in os.listdir(selected_folder_path):
    if gtfsrt_file.endswith('.gtfsrt'):
        gtfsrt_file_path = os.path.join(selected_folder_path, gtfsrt_file)
        feed = process_gtfsrt_file(gtfsrt_file_path)
        feeds.append(feed)
        if feed is not None:
            feeds.append(feed)
            if not first_file_processed:
                print("Contenu du premier fichier:")
                #print(feed)
                first_file_processed = True



#stop_time extraites de tous les fichiers
stop_time_updates_list = []

for gtfsrt_file in os.listdir(selected_folder_path):
    if gtfsrt_file.endswith('.gtfsrt'):
        gtfsrt_file_path = os.path.join(selected_folder_path, gtfsrt_file)
        feed = process_gtfsrt_file(gtfsrt_file_path)
        stop_time_updates = extract_stop_time_updates(feed)
        stop_time_updates_list.extend(stop_time_updates)


# Affichez le nombre de mises à jour de stop_time extraites de tous les fichiers
print(f"Nombre total de stop_time_updates extraites: {len(stop_time_updates_list)}")

# Affichez les 10 premières mises à jour de stop_time pour vérifier si elles ont été extraites correctement
print("Les 10 premières mises à jour de stop_time :")
for i, stop_time_update in enumerate(stop_time_updates_list[:10]):
    print(f"{i + 1}. Stop ID: {stop_time_update.stop_id}, Arrival Delay: {stop_time_update.arrival.delay}, Departure Delay: {stop_time_update.departure.delay}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dossier geospacialdata trouvé: /content/drive/MyDrive/Geospatial Data Science 2023
Contenu du premier fichier:
Nombre total de stop_time_updates extraites: 10332804
Les 10 premières mises à jour de stop_time :
1. Stop ID: 8844628, Arrival Delay: 0, Departure Delay: 60
2. Stop ID: 8844503, Arrival Delay: 120, Departure Delay: 180
3. Stop ID: 8844545, Arrival Delay: 0, Departure Delay: 0
4. Stop ID: 8844057, Arrival Delay: 0, Departure Delay: 0
5. Stop ID: 8844008, Arrival Delay: 60, Departure Delay: 120
6. Stop ID: 8844206, Arrival Delay: 0, Departure Delay: 0
7. Stop ID: 8844230, Arrival Delay: 0, Departure Delay: 0
8. Stop ID: 8844255, Arrival Delay: 0, Departure Delay: 0
9. Stop ID: 8844271, Arrival Delay: 0, Departure Delay: 0
10. Stop ID: 8842051, Arrival Delay: 0, Departure Delay: 0
